In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import numpy as np
np.set_printoptions(precision=3, suppress=True)

In [16]:
from quantum_masala.core import RealLattice, UPFv2Data, AtomBasis, Crystal

reallat = RealLattice.from_axes_alat(5.1070, [0.5, 0.5, 0.5], [-0.5, 0.5, 0.5], [-0.5, -0.5, 0.5])

fe_oncv = UPFv2Data.read('fe', './fe/Fe_ONCV_PBE-1.2.upf')
fe_atoms = AtomBasis(reallat, fe_oncv, 55.847, [[0., 0., 0.]])
crystal = Crystal(reallat, [fe_atoms])

In [17]:
recilat = crystal.recilat

print(f"alat = {reallat.alat}")
print("Axes of real space in cartesian")
for i, ai in enumerate(reallat.axes_cart):
    print(f"a({i+1}) = {ai}")
print()
print("Axes of real space in alat")
for i, ai in enumerate(reallat.axes_alat):
    print(f"a({i+1}) = {ai}")

print()
print('-'*40)

print(f"tpiba = {recilat.tpiba}")
print("Axes of real space in tpiba")
for i, bi in enumerate(recilat.axes_tpiba):
    print(f"b({i+1}) = {bi}")
print()
print("Axes of real space in cart")
for i, bi in enumerate(recilat.axes_cart):
    print(f"b({i+1}) = {bi}")
print()

alat = 5.107
Axes of real space in cartesian
a(1) = [2.554 2.554 2.554]
a(2) = [-2.554  2.554  2.554]
a(3) = [-2.554 -2.554  2.554]

Axes of real space in alat
a(1) = [0.5 0.5 0.5]
a(2) = [-0.5  0.5  0.5]
a(3) = [-0.5 -0.5  0.5]

----------------------------------------
tpiba = 1.2303084603837058
Axes of real space in tpiba
b(1) = [1. 0. 1.]
b(2) = [-1.  1.  0.]
b(3) = [ 0. -1.  1.]

Axes of real space in cart
b(1) = [1.23 0.   1.23]
b(2) = [-1.23  1.23  0.  ]
b(3) = [ 0.   -1.23  1.23]



In [19]:
print(recilat.recvec)

[[ 1.23 -1.23  0.  ]
 [ 0.    1.23 -1.23]
 [ 1.23  0.    1.23]]


In [32]:
from quantum_masala.core import KPoints, GSpace

grho = GSpace(crystal.recilat, 80 * np.sqrt(2))
print(grho.grid_shape)

(18, 18, 18)


In [33]:
PI = 3.141592
latvec = crystal.reallat.latvec
ecutwfc = 20
n1 = int(np.ceil(2 * np.sqrt(2 * ecutwfc) / PI * np.linalg.norm(latvec[:, 0])))
n2 = int(np.ceil(2 * np.sqrt(2 * ecutwfc) / PI * np.linalg.norm(latvec[:, 1])))
n3 = int(np.ceil(2 * np.sqrt(2 * ecutwfc) / PI * np.linalg.norm(latvec[:, 2])))

good_primes = [2, 3, 5]

n_new = []
for ni in [n1, n2, n3]:
    is_good = False
    while not is_good:
        n = ni
        for prime in good_primes:
            while n % prime == 0:
                n = n / prime
        if n == 1:
            is_good = True
        else:
            ni += 1
    n_new.append(ni)

print(n_new)

[18, 18, 18]
